# Lending Club Case Study

## Problem Statement

Lending Club company is the largest online loan marketplace, facilitating personal loans, business loans, and financing of medical procedures. Borrowers can easily access lower interest rate loans through a fast online interface. 

The company wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment. 

In [87]:
# import libraries

import numpy as np
import matplotlib.pyplot as plot
import seaborn as sea
import pandas as pd
import datetime as dt
import warnings

# suppress warnings
warnings.filterwarnings('ignore')

In [88]:
# Load Loan Data CSV into a pandas dataframe
loan_df = pd.read_csv('loan.csv')
loan_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


### Data Cleaning

1. Fix Rows and Columns
2. Fix missing values
3. Standardise Values
4. Fix Invalid Values
5. Filter Data

### Fix Rows

- **Remove summary, header, footer, extra rows**: There are no summary rows, there are no header or footer rows and there are no extra rows

- **Remove rows where loan status is `Current`**: If the loan status is `Current` then applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'. Therefore, these rows must be excluded from the analysis.

In [89]:
# Remove rows with loan status as current
original_rows,original_columns = loan_df.shape
loan_df = loan_df[loan_df['loan_status'] != "Current"]

current_rows,current_columns = loan_df.shape
print(f'Original Rows: {original_rows}, Current Rows: {current_rows}')
print(f'Number of Rows with status "Current" Removed: {original_rows - current_rows}')
print(f'Percentage of Rows Removed: {round((original_rows - current_rows)/original_rows*100,2)}%')
print(loan_df.shape)

Original Rows: 39717, Current Rows: 38577
Number of Rows with status "Current" Removed: 1140
Percentage of Rows Removed: 2.87%
(38577, 111)


### Fix Columns

Columns `'url', 'desc', 'emp_title', 'title'` are not a factor for loan disbursal decisions. Therefore, we can remove them

In [90]:
loan_df = loan_df.drop(['url', 'desc', 'emp_title', 'title'],  axis=1)
loan_df.shape

(38577, 107)

In [91]:
loan_df = loan_df.drop(['sub_grade', 'delinq_2yrs', 'earliest_cr_line', 
                          'inq_last_6mths', 'open_acc', 'pub_rec', 
                          'revol_bal', 'revol_util', 'total_acc', 
                          'out_prncp', 'out_prncp_inv', 'total_pymnt', 
                          'total_pymnt_inv', 'total_rec_prncp', 
                          'total_rec_int', 'total_rec_late_fee', 'recoveries', 
                          'collection_recovery_fee', 'last_pymnt_d', 
                          'last_pymnt_amnt', 'last_credit_pull_d', 
                          'application_type'], axis=1)
loan_df.shape


(38577, 85)

- Drop columns with more than 75% null or na values

In [92]:
threshold = 0.75 * len(loan_df)

loan_df = loan_df.dropna(axis='columns', thresh=threshold)
print(loan_df.shape)

(38577, 28)


- Drop columns with all zero values

In [93]:
loan_df = loan_df.loc[:, (loan_df != 0).any(axis=0)]
loan_df.shape

(38577, 26)

- Find unique values in each column

In [94]:
columns = loan_df.columns
num_unique_values = loan_df[columns].nunique()
num_unique_values

id                            38577
member_id                     38577
loan_amnt                       870
funded_amnt                    1019
funded_amnt_inv                8050
term                              2
int_rate                        370
installment                   15022
grade                             7
emp_length                       11
home_ownership                    5
annual_inc                     5215
verification_status               3
issue_d                          55
loan_status                       2
pymnt_plan                        1
purpose                          14
zip_code                        822
addr_state                       50
dti                            2853
initial_list_status               1
collections_12_mths_ex_med        1
policy_code                       1
chargeoff_within_12_mths          1
pub_rec_bankruptcies              3
tax_liens                         1
dtype: int64

As we can see above there are few columns which have constant values with value = 1 in all rows. We must remove these columns as they do not contribute to analysis and is considered redundant data.

In [95]:
columns_with_constant_values = columns[loan_df[columns].nunique() == 1]

loan_df = loan_df.drop(columns_with_constant_values, axis=1)
loan_df.shape

(38577, 20)

### Data Conversion

- Convert `term`, `int_rate`, `issue_d` to integer, float and date format respectively.

In [96]:
loan_df['term'] = loan_df['term'].str.extract('(\d+)').astype(int)

loan_df['int_rate'] = loan_df['int_rate'].str.extract('(\d+)').astype(float)

loan_df['issue_d'] = pd.to_datetime(loan_df['issue_d'], format='%b-%y')